In [6]:
from gurobipy import *
import numpy as np
import random
import time

# Paramètres du problème
n = 10  # nombre de sommets (V = {0, 1, …, n-1})
p = 1  # nombre de facilities (ressources)
V = range(n)
#E = [(0, 1), (1, 2), (2, 3), (3, 4)]  # arêtes
#E = [(0, 3), (1, 2)]
E = []
proba = 0.25

# Note : un graphe peut ne pas être connexe
for i in range(n):
    for j in range(i+1,n):
        pp = random.random()
        if (pp < proba):
            E.append((i,j))
print("Les arêtes du graphe :", E)

F = range(p)  # F = {0, 1, …, p-1}

def fct_w(n):
    return np.array([n - k for k in range(n)])

def fct_w_prime(w):
    n = len(w)
    w_prime = np.zeros(n)
    for k in range(n - 1):
        w_prime[k] = w[k] - w[k + 1]
    w_prime[n - 1] = w[n - 1]
    return w_prime

w = fct_w(n)          # Pour n = 5, w = [5, 4, 3, 2, 1]
w_prime = fct_w_prime(w)  # w_prime = [1, 1, 1, 1, 1]

# Matrice des coûts aléatoires
c = np.random.randint(1, 1000, size=(n, p))
#c = np.array([[1,3,3],[2,4,2],[3,2,3],[3,4,4],[1,3,2]])
#print("La matrice des coûts c :")
#for i in range(n):
#    print("i =", i, "|", c[i])

def constr_model(c, w_prime, n, p):
    model = Model(name="vertex cover OWA")
    # On résout la relaxation linéaire : x est déclaré CONTINU, borné entre 0 et 1.
    b = model.addVars(n, n, vtype=GRB.CONTINUOUS, name="b", lb=0)
    r = model.addVars(n, name='r', vtype=GRB.CONTINUOUS)
    x = model.addVars(n, p, vtype=GRB.CONTINUOUS, name="x", lb=0, ub=1)
    
    # Contrainte c3 (voir votre formulation) :
    model.addConstrs((r[k1] + b[i, k1] >= sum(c[i, k] * x[i, k] for k in F)
                        for i in range(n) for k1 in range(n)), name='c3')
    # Pour chaque arête, et pour chaque facility k, on impose x[i,k] + x[j,k] >= 1.
    for (i, j) in E:
        for k in F:
            model.addConstr(x[i, k] + x[j, k] >= 1, name="edge_{}_{}_{}".format(i, j, k))
    
    # Fonction objectif (en OWA, ici la version obtenue avec w_prime)
    obj_fn = sum(w_prime[k1] * ((k1 + 1) * r[k1] + sum(b[i, k1] for i in range(n)))
                 for k1 in range(n))
    model.setObjective(obj_fn, GRB.MINIMIZE)
    model.Params.OutputFlag = 0
    return model, x

# Construction du modèle initial en LP relaxé.
start = time.time()
model, x = constr_model(c, w_prime, n, p)

def iterative_rounding(model, x, E, F):
    iteration = 0
    while True:
        print("\n---- Itération", iteration, "----")
        # On résout la relaxation
        model.optimize()
        if model.Status != GRB.OPTIMAL:
            print("La résolution n'est pas optimale.")
            break
        
        changed = False
        # Sauvegarde des valeurs de x dans la solution courante pour faciliter les comparaisons
        sol = {}
        for i in range(n):
            for k in F:
                sol[(i, k)] = x[i, k].X

        # Pour chaque contrainte d'arête et pour chaque facility
        for (i, j) in E:
            for k in F:
                # Si pour cette contrainte l'une des deux variables est déjà fixée à 1, on passe à l'itération suivante
                if abs(x[i, k].LB - 1.0) < 1e-5 or abs(x[j, k].LB - 1.0) < 1e-5:
                    continue

                xi_val = sol[(i, k)]
                xj_val = sol[(j, k)]
                # On fixe à 1 la variable dont la valeur est supérieure.
                if xi_val >= xj_val:
                    #print("Fixation de x[{}, {}] passant de {:.3f} à 1 (arête ({}, {}))".format(i, k, xi_val, i, j))
                    x[i, k].LB = 1.0
                    x[i, k].UB = 1.0
                    changed = True
                else:
                    #print("Fixation de x[{}, {}] passant de {:.3f} à 1 (arête ({}, {}))".format(j, k, xj_val, i, j))
                    x[j, k].LB = 1.0
                    x[j, k].UB = 1.0
                    changed = True

        model.update()
        iteration += 1
        # S'il n'y a eu aucun changement dans cette boucle, on sort du while.
        if not changed:
            break
    print("Arrondi itératif terminé après {} itérations.".format(iteration))

# Lancement de l'arrondi itératif
iterative_rounding(model, x, E, F)
end = time.time()
time_approx = end-start

# Affichage de la solution finale
print('Valeur de la fonction objectif: %f' % model.objVal)
print("Temps pris pour algo approché :", time_approx)
print("\nSolution finale :")
for i in range(n):
    for k in F:
        print("x[{}, {}] = {:.3f}".format(i, k, x[i, k].X))

Les arêtes du graphe : [(0, 2), (0, 7), (0, 8), (0, 9), (1, 2), (1, 4), (1, 9), (2, 8), (2, 9), (3, 5), (4, 6), (4, 8), (5, 9)]

---- Itération 0 ----

---- Itération 1 ----
Arrondi itératif terminé après 2 itérations.
Valeur de la fonction objectif: 60542.000000
Temps pris pour algo approché : 0.00689244270324707

Solution finale :
x[0, 0] = 1.000
x[0, 1] = 1.000
x[0, 2] = 1.000
x[1, 0] = 0.000
x[1, 1] = 1.000
x[1, 2] = 1.000
x[2, 0] = 1.000
x[2, 1] = 1.000
x[2, 2] = 1.000
x[3, 0] = 1.000
x[3, 1] = 0.000
x[3, 2] = 1.000
x[4, 0] = 1.000
x[4, 1] = 1.000
x[4, 2] = 1.000
x[5, 0] = 0.000
x[5, 1] = 1.000
x[5, 2] = 0.000
x[6, 0] = 0.000
x[6, 1] = 0.000
x[6, 2] = 0.000
x[7, 0] = 0.000
x[7, 1] = 0.000
x[7, 2] = 0.000
x[8, 0] = 0.000
x[8, 1] = 0.000
x[8, 2] = 0.000
x[9, 0] = 1.000
x[9, 1] = 0.000
x[9, 2] = 1.000
